In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS, add_constant

import matplotlib.pyplot as plt

## Load data and build test/train split

In [2]:
data = pd.read_csv("./data/RRCA_baseflow.csv")

In [3]:
def test_train_split():
    data = data.sample(frac=1)
    split = int(0.8 * len(data))

    train = data.iloc[:split, :]
    test = data.iloc[split:, :]

    train.to_csv("./data/train.csv", index=None)
    test.to_csv("./data/test.csv", index=None)

# test_train_split()

In [4]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

split = int(0.8 * len(train))
train = train.sample(frac=1)

validation = train.iloc[split:, :]
validation_y = validation[["Observed"]]
validation_x = validation.drop(["Observed"], axis=1)

train = train.iloc[:split, :]
train_y = train[["Observed"]]
train_x = train.drop(["Observed"], axis=1)

## Explore data

In [12]:
# print(f"Columns: {', '.join(data.columns.values)}")
# print()

# for header in data.columns:
#     print(header)
#     print(f"std: {data[[header]].values.flatten().std()}")
#     print(f"avg: {data[[header]].values.flatten().mean()}")
#     print(f"min: {data[[header]].values.flatten().min()}")
#     print(f"max: {data[[header]].values.flatten().max()}")
#     print()

## SKLearn Linear Regression

In [13]:
model = LinearRegression(fit_intercept=True, n_jobs=6)
model.fit(train_x, train_y)

prediction_y = model.predict(validation_x).flatten()
actual_y = validation_y.to_numpy().flatten()

mean_squared_error = 0.
mean_log_error = 0.
for index in range(len(prediction_y)):
    mean_squared_error += (prediction_y[index] - actual_y[index]) ** 2
    mean_log_error += np.log(np.abs(prediction_y[index] - actual_y[index]))
mean_squared_error /= len(prediction_y)
mean_log_error /= len(prediction_y)

r_squared = model.score(validation_x, validation_y)

print(f"MSE: {mean_squared_error:10.4f}")
print(f"MLE: {mean_log_error:10.4f}")
print(f"R^2: {r_squared:10.4f}")

MSE:  2246.2538
MLE:     2.7883
R^2:     0.2573


## Statsmodels Linear Regression

In [65]:
train_x2 = add_constant(train_x)
model = OLS(train_y, train_x2)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Observed   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.233
Method:                 Least Squares   F-statistic:                     434.6
Date:                Tue, 10 Mar 2020   Prob (F-statistic):               0.00
Time:                        22:56:54   Log-Likelihood:                -53166.
No. Observations:                9977   AIC:                         1.063e+05
Df Residuals:                    9969   BIC:                         1.064e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               1661.3416    113.541     14.632      0.000    1438.778    1883.906
Date                  -0.0016   8.33e-05    -19.590      0.000      -0.002      -0.001
Segment_id             0.3524      0.010     35.451      0.000       0.333       0.372
x                   6.247e-06   1.81e-06      3.445      0.001    2.69e-06     9.8e-06
y                  -3.702e-05    6.7e-06     -5.521      0.000   -5.02e-05   -2.39e-05
Evapotranspiration    -0.3781      0.203     -1.860      0.063      -0.777       0.020
Precipitation          1.7179      0.057     30.232      0.000       1.607       1.829
Irrigation_pumping     6.3056      2.178      2.895      0.004       2.036      10.575
==============================================================================
Omnibus:                    11651.311   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1399967.498
Skew:                           6.162   Prob(JB):                         0.00
Kurtosis:                      59.708   Cond. No.                     3.33e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.33e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Statsmodels Linear Regression - Differing zones

In [63]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

split = int(0.8 * len(train))
train = train.sample(frac=1)


def ln_reg(data):

    y = data[["Observed"]]
    x = data.drop(["Observed", "Segment_id", "x", "y"], axis=1)

    x2 = add_constant(x)
    model = OLS(y, x2)
    results = model.fit()
    print(f"Area: {'':15s} {int(data.iloc[0][['Segment_id']].values)}")
    print(f"R^2:  {'':15s} {results.rsquared:.7f}")
    print(results.pvalues)
    print()

train.groupby("Segment_id").apply(ln_reg)

Area:                 40
R^2:                  0.4070359
const                 1.085925e-06
Date                  6.655312e-05
Evapotranspiration    6.987567e-02
Precipitation         6.458253e-02
Irrigation_pumping    4.140237e-11
dtype: float64

Area:                 51
R^2:                  0.0331012
const                 0.000304
Date                  0.000259
Evapotranspiration         NaN
Precipitation              NaN
Irrigation_pumping         NaN
dtype: float64

Area:                 53
R^2:                  0.0857821
const                 0.876543
Date                  0.902772
Evapotranspiration    0.000125
Precipitation         0.487609
Irrigation_pumping    0.110979
dtype: float64

Area:                 55
R^2:                  0.2100320
const                 9.922176e-10
Date                  6.698476e-10
Evapotranspiration    3.244546e-04
Precipitation         1.168079e-03
Irrigation_pumping    2.360019e-01
dtype: float64

Area:                 56
R^2:                  0

""
